# Описание проекта

В данном ноутбуке выполнен файнтьюн bert подобных моделей для задачи token classification в частности NEr для круса MLOPS от ODS
Модель построена самым простым образом, т.к. основная задача курса это проведение исследований и разворачивание моделей в проде. 

# Загружаем данные 

In [1]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import evaluate
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch
from transformers import pipeline

/home/ivan/miniconda3/envs/mlops/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Для разработки модели возьмем классический датасет conll2003

In [83]:
raw_datasets = load_dataset("conll2003")

Reusing dataset conll2003 (/home/ivan/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 550.72it/s]


Посмотрим на структуру данных:

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
raw_datasets['train']['ner_tags'][0]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [5]:
raw_datasets['train']['tokens'][0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [6]:
raw_datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

# Обработка данных 

Посмотрим на сами данные:

In [7]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
words = raw_datasets['train'][0]['tokens']
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2) 

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [10]:
words = raw_datasets['train'][4]['tokens']
labels = raw_datasets["train"][4]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2) 

Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
B-LOC   O  O              O  O   B-ORG    I-ORG O  O          O         B-PER  I-PER     O    O  O         O         O      O   O         O    O         O     O    B-LOC   O     O   O          O      O   O       O 


In [11]:
words = raw_datasets['train'][0]['tokens']
pos_names = raw_datasets['train'].features['pos_tags'].feature.names
labels = raw_datasets["train"][0]["pos_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = pos_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2) 

EU  rejects German call to boycott British lamb . 
NNP VBZ     JJ     NN   TO VB      JJ      NN   . 


Для удобства рассмотрим дистилированную модель bert-tiny

In [2]:
model_checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [13]:
inputs = tokenizer(raw_datasets["train"][4]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'germany',
 "'",
 's',
 'representative',
 'to',
 'the',
 'european',
 'union',
 "'",
 's',
 'veterinary',
 'committee',
 'werner',
 'z',
 '##wing',
 '##mann',
 'said',
 'on',
 'wednesday',
 'consumers',
 'should',
 'buy',
 'sheep',
 '##me',
 '##at',
 'from',
 'countries',
 'other',
 'than',
 'britain',
 'until',
 'the',
 'scientific',
 'advice',
 'was',
 'clearer',
 '.',
 '[SEP]']

In [14]:
inputs.word_ids()

[None,
 0,
 1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 7,
 8,
 9,
 10,
 11,
 11,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 18,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 None]

Т.к. при токенизации некоторые слова могут разбиваться на несколько токенов, то необходимо привести в соответствие, чтобы новым токенам был приведен label слова.

In [15]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [16]:
labels = raw_datasets["train"][4]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]


In [17]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [18]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Loading cached processed dataset at /home/ivan/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-7f7b7bdf1cf63085.arrow
Loading cached processed dataset at /home/ivan/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-2397cfa85b91cecd.arrow
100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 12.44ba/s]


In [19]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(3)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100,    5,    0,    0,    0,    0,    0, -100, -100, -100, -100]])

# Метрики 

Для анализа метрик воспользуемся библиотекой seqeval

In [84]:
metric = evaluate.load("seqeval")

In [85]:
metric

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [86]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# Подготовливаем данные для модели

In [24]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [25]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

#  Модель

In [26]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [37]:
num_warmup_steps = 0
num_train_epochs = 3
output_dir = './models'
batch_size = 8

In [28]:
model_checkpoint

'prajjwal1/bert-tiny'

In [29]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

In [30]:
model.config.num_labels

9

In [31]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [32]:
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [33]:
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)


In [42]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

#     # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + f'/epoch_{epoch}', save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
#         repo.push_to_hub(
#             commit_message=f"Training in progress epoch {epoch}", blocking=False
#         )


  9%|███▍                                    | 456/5268 [00:11<02:04, 38.67it/s]


 17%|██████▉                                 | 918/5268 [00:23<01:57, 37.01it/s]


 26%|██████████                             | 1357/5268 [00:34<01:48, 36.05it/s]


 33%|█████████████                          | 1758/5268 [00:47<14:17,  4.09it/s]

epoch 0: {'precision': 0.629585997980478, 'recall': 0.5591928251121077, 'f1': 0.5923052564914503, 'accuracy': 0.919424277566842}


...


 75%|█████████████████████████████▍         | 3973/5268 [01:47<00:33, 39.23it/s]


 84%|████████████████████████████████▉      | 4446/5268 [01:59<00:20, 40.58it/s]


 94%|████████████████████████████████████▍  | 4929/5268 [02:10<00:08, 41.13it/s]


100%|███████████████████████████████████████| 5268/5268 [02:19<00:00, 41.43it/s]

epoch 2: {'precision': 0.629585997980478, 'recall': 0.5591928251121077, 'f1': 0.5923052564914503, 'accuracy': 0.919424277566842}


#  Анализ результатов

Протестируем модель на тестовых данных:

In [43]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], 
    collate_fn=data_collator,
    batch_size=8
)

Загружаем модель из сохраненных данных от последней эпохи:

In [44]:
model = AutoModelForTokenClassification.from_pretrained('./models/epoch_2')

In [77]:
model.eval()
all_predictions = []
all_test = []
for batch in test_dataloader:
    with torch.no_grad():
        outputs = model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    # Necessary to pad predictions and labels for being gathered
    predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(predictions)
    labels_gathered = accelerator.gather(labels)

    true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=true_predictions, references=true_labels)
    
    all_test.extend(true_labels)
    all_predictions.extend(true_predictions)
    
results = metric.compute()
print(
    {
        key: results[f"overall_{key}"]
        for key in ["precision", "recall", "f1", "accuracy"]
    },
)

{'precision': 0.5931010207673354, 'recall': 0.5089865579217641, 'f1': 0.5478338616597578, 'accuracy': 0.9108815757866923}


In [78]:
results 

{'LOC': {'precision': 0.6947743467933492,
  'recall': 0.6148187073042565,
  'f1': 0.6523557290214665,
  'number': 1903},
 'MISC': {'precision': 0.4410112359550562,
  'recall': 0.47289156626506024,
  'f1': 0.45639534883720934,
  'number': 664},
 'ORG': {'precision': 0.3835039133052378,
  'recall': 0.3607021517553794,
  'f1': 0.3717537204552086,
  'number': 1766},
 'PER': {'precision': 0.7686153846153846,
  'recall': 0.5458916083916084,
  'f1': 0.6383848709430104,
  'number': 2288},
 'overall_precision': 0.5931010207673354,
 'overall_recall': 0.5089865579217641,
 'overall_f1': 0.5478338616597578,
 'overall_accuracy': 0.9108815757866923}

In [82]:
all_test[1]

['B-PER', 'I-PER', 'I-PER', 'I-PER']

In [81]:
all_predictions[1]

['B-PER', 'I-PER', 'I-PER', 'I-PER']

In [74]:
test_0.tokens()

['[CLS]',
 'soccer',
 '-',
 'japan',
 'get',
 'lucky',
 'win',
 ',',
 'china',
 'in',
 'surprise',
 'defeat',
 '.',
 '[SEP]']